# Confidence intervals

Import python modules

In [1]:
import matplotlib.pylab as plt
import numpy as np
from scipy import stats

### Table of Contents

1. [Background](#background)<br><br>
2. [Single population mean CI](#popmean)<br>
    2a. [if population sigma is unknown -> t distribution](#t_ci)<br>
    2b. [if population sigma is known -> normal distribution](#norm_ci)<br>
    2c. [sampling (n) required for desired CI](#norm_ci_n)<br><br>
3. [Population proportion CI](#bi_ci)<br>
    3a. [using a binomial approximated as normal](#bi_ci)<br>
    3b. [+4 correction](#plus4)<br>
    3c. [sampling (n) required for desired confidence level](#norm_ci_n_pro)<br>

### Background<a class="anchor" id="background"></a>

A **confidence interval (CI)** is a statistical inference, range estimation for the population mean. The population mean falls within the CI with some probability, the **confidence level (CL)**. More accurately, the CL is the percent of confidence intervals that would contain the true population parameter if repeated samples are taken.

### Single population mean CI<a class="anchor" id="popmean"></a>

#### if the population standard deviation is unknown <a class="anchor" id="t_ci"></a>

When the population standard deviation is unknown (as is usually the case), a Students t test should be used to calculate a confidence interval for the mean.

The confidence interval is: $\bar{x} ± EBM$<br><br>
The EBM (error bound of the mean) is: $$\large(t_{df,\LARGE\frac{\alpha}{2}})( \frac{s} {\sqrt{n}})$$

s = sample standard deviation<br>
*note: calculated using $\sqrt{n-1}$ in demoninator*<br>

t$_{df}$ = A Students t distribution for degrees of freedom (n-1)<br>
t$_{(\alpha/2)}$ = t-score on t$_{df}$ for (1 - CL) / 2.<br>
*note: divide by two for two-tailed test only*

In [2]:
# Illowsky - Example 8.8
data = [8.6, 9.4, 7.9, 6.8, 8.3, 7.3, 9.2, 9.6, 8.7, 11.4, 10.3, 5.4, 8.1, 5.5, 6.9]
cl = 0.95

x_bar = np.mean(data) # sample mean
s = np.std(data, ddof=1) # sample standard deviation, ddof=1 -> denominator sqrt(n-1)
n = len(data)
alpha = 1 - cl
t_score = (stats.t.ppf((1 - (alpha / 2)), n - 1)) 
ebm = t_score * (s / np.sqrt(n))
ci_low = x_bar - ebm
ci_high = x_bar + ebm

print('The {0} percent CI is {1:.2f} to {2:.2f}'.format(cl * 100, ci_low, ci_high))
print('From a sample mean of {0:.2f} and sample standard deviation {1:.2f}'.format(x_bar, s))
print('')

# or go straight to getting bounds with t.interval
ci_low, ci_high = stats.t.interval(cl, n - 1, x_bar, (s / np.sqrt(n)))
print('The {0} percent CI is {1:.2f} to {2:.2f} found with scipy t.interval'.\
      format(cl * 100, ci_low, ci_high))

The 95.0 percent CI is 7.30 to 9.15
From a sample mean of 8.23 and sample standard deviation 1.67

The 95.0 percent CI is 7.30 to 9.15 found with scipy t.interval


In [3]:
# Illowsky - Example 8.9 
#   same concept as example 8.8 above
data = [ 79, 145, 147, 160, 116, 100, 159, 151, 156, 126,
        137,  83, 156,  94, 121, 144, 123, 114, 139,  99]
cl = 0.90

x_bar = np.mean(data) # sample mean
s = np.std(data, ddof=1) # sample standard deviation
n = len(data)
ci_low, ci_high = stats.t.interval(cl, n - 1, x_bar, (s / np.sqrt(n)))

print('The {0} percent CI is {1:.2f} to {2:.2f} (found with scipy t.interval)'.\
      format(cl * 100, ci_low, ci_high))

The 90.0 percent CI is 117.41 to 137.49 (found with scipy t.interval)


#### if the population standard deviation is known<a class="anchor" id="norm_ci"></a>

When the population standard deviation is known (this is rare), the normal distribution can be used to calculate a confidence interval for the mean.

The confidence interval is $\bar{x}$ ± EBM<br>
The EBM (error bound of the mean) is: $$\large(z_{\LARGE\frac{\alpha}{2}})( \frac{\sigma} {\sqrt{n}})$$
z($\alpha$/2) = z-score on a normal distribution for (1 - CL) / 2<br>
note: dividing by two for two-tailed test

In [4]:
#   Illowsky - Example 8.3
x_bar = 1.024  # sample mean
sigma = 0.337  # known population standard deviation
cl = 0.98
n = 30

alpha = 1 - cl
z = (stats.norm.ppf(1 - (alpha / 2), 0, 1)) 
ebm = z * sigma / np.sqrt(n)
ci_low = x_bar - ebm
ci_high = x_bar + ebm

print('The {0} percent CI is {1:.2f} to {2:.2f}'.format(cl * 100, ci_low, ci_high))
print('From a sample mean of {0} from a population with known standard deviation {1}'.format(x_bar, sigma))
print()

# or go straight to getting bounds with norm.interval
ci_low, ci_high = stats.norm.interval(cl, x_bar, (sigma / np.sqrt(n)))
print('The {0} percent CI is {1:.2f} to {2:.2f} found with scipy norm.interval'.\
      format(cl * 100, ci_low, ci_high))

The 98.0 percent CI is 0.88 to 1.17
From a sample mean of 1.024 from a population with known standard deviation 0.337

The 98.0 percent CI is 0.88 to 1.17 found with scipy norm.interval


#### Number of samples required <a class="anchor" id="norm_ci_n"></a>

The number of samples required for a desired confidence level and interval can be determined in advance.

$$n = \frac{z^2 \sigma^2}{EBM^2}$$

In [5]:
# Illowsky - example 8.7
ebm = 2
conf = 0.95
sigma = 15

alpha = (1 - conf)
z = (stats.norm.ppf((alpha / 2)))
n = np.ceil((z ** 2 * sigma ** 2) / (ebm ** 2))
print('To achieve a range of +/-{0} with a confidence of {1} percent would require {2:.0f} samples'.\
      format(ebm, conf * 100, n))

To achieve a range of +/-2 with a confidence of 95.0 percent would require 217 samples


### Population Proportion CIs<a class="anchor" id="bi_ci"></a>

When estimating a population proportion, obtain the standard deviation from binomial modeled as normal distribution, dividing by n to account for the proportion.

The confidence interval is $\bar{x}$ ± EBP<br>
EBP = Error bound of the proportion = z($\alpha$/2) \* ($\sqrt{p' q' / n}$)<br>
p' = proportion of success estimated from the sample = p / n<br>
q' = 1 - p'<br>
z($\alpha$/2) = z-score on a normal distribution for (1 - CL) / 2<br>
note: dividing by two for two-tailed test<br>

In [6]:
# Illowsky - Example 8.10 
n = 500
p = 421
cl = 0.95

# Check that number of success and failures are both > 5
if p <= 5 or (n - p) <= 5:
    raise UserWarning('The number of success or failures is too low for an accurate confidence interval')

# Calculate confidence interval
p_prime = p / n
alpha = 1 - cl
z_score = stats.norm.ppf(1 - (alpha / 2))
s = np.sqrt((p_prime * (1 - p_prime)) / n)
ebp = z_score * s
ci_low = p_prime - ebp
ci_high = p_prime + ebp
print('The {0} percent CI is {1:.3f} to {2:.3f}'.format(cl * 100, ci_low, ci_high))
print()

# Scipy confidence interval function
ci_low, ci_high = (stats.norm.interval(cl, p_prime, s))
print('The {0} percent CI is {1:.3f} to {2:.3f} (found with scipy norm.interval)'.\
      format(cl * 100, ci_low, ci_high))

The 95.0 percent CI is 0.810 to 0.874

The 95.0 percent CI is 0.810 to 0.874 (found with scipy norm.interval)


#### "Plus Four" CI<a class="anchor" id="plus4"></a>

A method for reducing error when sample size is small (n is small) or the same is imbalanced (p or q is small). It involves adding four phantom observations to the sample, two success and two failure (n + 4, p + 2)

Note, plus four should not be used for confidence < 90% or sample size < 10.


In [7]:
# Illowsky - Example 8.12
#  low p
n = 25     # number of samples
p = 6     # number of successes
cl = 0.95  # confidence level

# Without plus 4 method
p_prime = p / n   # probability of success
alpha = 1 - cl
z_score = stats.norm.ppf(1 - (alpha / 2))
s = np.sqrt(p_prime * (1 - p_prime) / n)
ebp = z_score * s
ci_lower = p_prime - ebp
ci_upper = p_prime + ebp
print('For p = {0}, n = {1}:'.format(n , p))
print('  The {0} percent interval without the plus four method  {1:.3f} to {2:.3f}'.\
      format(cl * 100, ci_lower, ci_upper))

# Applying +4 method
n += 4     
p += 2

p_prime = p / n   # probability of success
alpha = 1 - cl
z_score = stats.norm.ppf(1 - (alpha / 2))
s = np.sqrt(p_prime * (1 - p_prime) / n)
ebp = z_score * s
ci_lower = p_prime - ebp
ci_upper = p_prime + ebp

print('  The {0} percent interval using the plus four method:   {1:.3f} to {2:.3f}'.\
      format(cl * 100, ci_lower, ci_upper))
print('  Low p, +4 pulls up range')
# using scipy norm.interval
#ci_lower, ci_upper = stats.norm.interval(cl, p_prime, s)
#print('The {0} percent interval using the plus four method:   {1:.3f} to {2:.3f}'
#      ' (found with scipy norm.interval)'.\
#      format(cl * 100, ci_lower, ci_upper))

For p = 25, n = 6:
  The 95.0 percent interval without the plus four method  0.073 to 0.407
  The 95.0 percent interval using the plus four method:   0.113 to 0.439
  Low p, +4 pulls up range


In [8]:
n = 25     # number of samples
p = 23     # number of successes
cl = 0.95  # confidence level

# Without plus 4 method
p_prime = p / n   # probability of success
alpha = 1 - cl
z_score = stats.norm.ppf(1 - (alpha / 2))
s = np.sqrt(p_prime * (1 - p_prime) / n)
ebp = z_score * s
ci_lower = p_prime - ebp
ci_upper = p_prime + ebp
print('For p = {0}, n = {1}:'.format(p , n))
print('  The {0} percent interval without the plus four method  {1:.3f} to {2:.3f}'.\
      format(cl * 100, ci_lower, ci_upper))

# Applying +4 method
n += 4     
p += 2

p_prime = p / n   # probability of success
alpha = 1 - cl
z_score = stats.norm.ppf(1 - (alpha / 2))
s = np.sqrt(p_prime * (1 - p_prime) / n)
ebp = z_score * s
ci_lower = p_prime - ebp
ci_upper = p_prime + ebp

print('  The {0} percent interval using the plus four method:   {1:.3f} to {2:.3f}'.\
      format(cl * 100, ci_lower, ci_upper))
print('  High p, +4 pushes down range')

For p = 23, n = 25:
  The 95.0 percent interval without the plus four method  0.814 to 1.026
  The 95.0 percent interval using the plus four method:   0.737 to 0.988
  High p, +4 pushes down range


#### number of samples required (for proportion) <a class="anchor" id="bi_ci_n_pro"></a>

The number of samples required for a desired confidence level and interval of a proportion can be estimated in advance.

$$\large 
n = \frac{z^2 p q} {EBP^2}
$$

*note for pq use worst case for required sampling = 0.5 $*$ 0.5 = 0.25*

In [9]:
# Illowsky - Example 8.14
ebp = 0.03
conf = 0.90

alpha = (1 - conf)
z = (stats.norm.ppf((alpha / 2), 0, 1))
n = np.ceil((z ** 2 * 0.25) / (ebp ** 2))
print('The estimated number of samples to reach an error bound of the proportion of {0} '
      'with {1} percent confidence is {2:.0f}.'.format(ebp, conf * 100, n))

The estimated number of samples to reach an error bound of the proportion of 0.03 with 90.0 percent confidence is 752.


## Sources<a class="anchor" id="sources"></a>

Illowsky, Barbara; Dean, Susan. Introductory Statistics. OpenStax College. Kindle Edition
https://openstax.org/details/introductory-statistics

SciPy 1.0.0 Release Notes: https://docs.scipy.org/doc/scipy/reference/index.html